In [46]:
!pip install vecstack

from vecstack import stacking
import pandas as pd
import numpy as np


from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVR

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive



Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [47]:
trainfile = r'/gdrive/My Drive/CIS508/TeamAssignment1/Colab/train.csv'
P_trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/gdrive/My Drive/CIS508/TeamAssignment1/Colab/test.csv'
testData = pd.read_csv(testfile)  #creates a dataframe

trainData=P_trainData.iloc[:,:-1].copy()
print(trainData.shape)

print(testData.shape)
trainData.head()


(1460, 80)
(1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [48]:
testData.head(20)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
5,1466,60,RL,75.0,10000,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1993,1994,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,763.0,763.0,GasA,Gd,Y,SBrkr,763,892,0,1655,0.0,0.0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1993.0,Fin,2.0,440.0,TA,TA,Y,157,84,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
6,1467,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,1Story,6,7,1992,2007,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,Gd,PConc,Gd,TA,No,ALQ,935.0,Unf,0.0,233.0,1168.0,GasA,Ex,Y,SBrkr,1187,0,0,1187,1.0,0.0,2,0,3,1,TA,6,Typ,0,NaN,Attchd,1992.0,Fin,2.0,420.0,TA,TA,Y,483,21,0,0,0,0,NaN,GdPrv,Shed,500,3,2010,WD,Normal
7,1468,60,RL,63.0,8402,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1998,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,789.0,789.0,GasA,Gd,Y,SBrkr,789,676,0,1465,0.0,0.0,2,1,3,1,TA,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,393.0,TA,TA,Y,0,75,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
8,1469,20,RL,85.0,10176,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,1Story,7,5,1990,1990,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,PConc,Gd,TA,Gd,GLQ,637.0,Unf,0.0,663.0,1300.0,GasA,Gd,Y,SBrkr,1341,0,0,1341,1.0,0.0,1,1,2,1,Gd,5,Typ,1,Po,Attchd,1990.0,Unf,2.0,506.0,TA,TA,Y,192,0,0,0,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal
9,1470,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,4,5,1970,1970,Gable,CompShg,

In [49]:
#DROP COLUMNS WITH LOTS OF MISSING VALUES===============================
#CAN ALSO DROP ROWS WITH LOTS OF MISSING VALUES
#Combine Train data and test data first so that the SAME COLUMNS are DROPPED in each
combined_Data = pd.concat([trainData, testData], keys=[0,1])
percent=int(0.6*(combined_Data.shape[0]))
print(percent)
#Drop columns that have less than "thresh" number of non_Nans
Processing = combined_Data.dropna(thresh=percent,axis=1)
print(Processing.shape)


1751
(2919, 75)


In [50]:
#NOW IMPUTE MISSING VALUES FOR THE OTHER COLUMNS=========================
#IMPUTE (SUBSTITUTE) MEAN VALUES FOR NaN IN NUMERIC COLUMNS 
numeric = Processing.select_dtypes(include=['int','float64']).columns
for num in numeric:
  Processing[num] = Processing[num].fillna(Processing[num].mean())

#IMPUTE (SUBSTITUTE) MODE VALUES FOR NaN IN CATEGORICAL COLUMNS
train_cat_cols = Processing.select_dtypes(exclude=['int','float64']).columns#selecting the categorical columns
for colss in train_cat_cols:
  Processing[colss] = Processing[colss].fillna(Processing[colss].mode()[0])

print(Processing.head(20))



      Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape LandContour Utilities LotConfig LandSlope Neighborhood Condition1 Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces GarageType  GarageYrBlt GarageFinish  GarageCars  GarageArea GarageQual GarageCond PavedDrive  WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  MiscVal  MoSold  YrSold SaleType SaleCondition
0 0    1          60       RL    65.000000     8450   Pave      Reg         Lvl    AllPub    Inside       Gtl      CollgCr       Norm       Norm     1Fam   

In [51]:
#Check final state of nulls in the columns
Processing.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinSF1       0
BsmtFinType2     0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
KitchenQual 

In [0]:
#Export Processed data 
F_Train = Processing.xs(0)
F_Test = Processing.xs(1)
F_Train.to_csv(r'/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Preprocess_Train.csv',index=None)
F_Test.to_csv(r'/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Preprocess_Test.csv',index=None)

In [53]:
#Import Processed data and check it
trainfile = r'/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Preprocess_Train.csv'
trainData = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Preprocess_Test.csv'
testData = pd.read_csv(testfile)

print(trainData.shape)
print(testData.shape)


(1460, 75)
(1459, 75)


In [54]:
trainData.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal


In [55]:
testData.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,0,1,2010,WD,Normal


In [56]:
#Copy Train data excluding target and unnamed column
trainData_Copy = trainData.iloc[:,1:].copy()
#Copy Test data excluding ID column 
testData_Copy = testData.iloc[:,1:].copy()

print(trainData_Copy.head())
print(testData_Copy.head())

   MSSubClass MSZoning  LotFrontage  LotArea Street LotShape LandContour Utilities LotConfig LandSlope Neighborhood Condition1 Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces GarageType  GarageYrBlt GarageFinish  GarageCars  GarageArea GarageQual GarageCond PavedDrive  WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  MiscVal  MoSold  YrSold SaleType SaleCondition
0          60       RL         65.0     8450   Pave      Reg         Lvl    AllPub    Inside       Gtl      CollgCr       Norm       Norm     1Fam     2Story      

In [57]:
#List of Categorical Features
#categoricalFeatures = ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']
train_cat_cols = trainData_Copy.select_dtypes(exclude=['float','int']).columns
print(train_cat_cols.shape)
print(train_cat_cols)

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([trainData_Copy, testData_Copy], keys=[2,3])
print(combined_Data.shape)

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=train_cat_cols)
print(combined_Data.shape)

#Separate Train data and test data
X_Train = combined_Data.xs(2)
X_Test = combined_Data.xs(3)

#X_Test.head()
print(X_Train.shape)
print(X_Test.shape)

print(X_Train.head)
print(X_Test.head)

#Select just Target Column
Y_Train = P_trainData.iloc[:, -1]
print(Y_Train.head)

(38,)
Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition'], dtype='object')
(2919, 74)
(2919, 270)
(1460, 270)
(1459, 270)
<bound method NDFrame.head of       MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  TotRmsAbvGrd  Fireplaces  GarageYrBlt  GarageCars  GarageArea  WoodDeckSF  OpenPorchSF  EnclosedPorc

In [58]:
#Decision Tree Regressor ========================================================================
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeRegressor()
clf.fit(X_Train, Y_Train)
clf_predict_train=clf.predict(X_Train)

#clf.feature_importances_
mean_squared_error(Y_Train,clf_predict_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(Y_Train,clf_predict_train)))

#test data
clf_predict_Test=clf.predict(X_Test)

pred=pd.DataFrame(clf_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_DF_DT.csv',index=None)




RMSE (training) for Decision Tree:  0.000000
   SalePrice
0   129000.0
1   155000.0
2   185000.0
3   181000.0
4   212000.0
     Id  SalePrice
0  1461   129000.0
1  1462   155000.0
2  1463   185000.0
3  1464   181000.0
4  1465   212000.0


In [59]:
#Hyperparameter tuning done for decision tree 
parameters={'min_samples_split' : range(10,100,10),'min_samples_leaf': range(2,8,1)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=10)
clf_random.fit(X_Train,Y_Train)
rand_parm=clf_random.best_params_
print(rand_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTree 
clf = DecisionTreeRegressor(**rand_parm)
clf.fit(X_Train,Y_Train)
clf_predict_train = clf.predict(X_Train)

#clf.feature_importances_
mean_squared_error(Y_Train,clf_predict_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(Y_Train,clf_predict_train)))

#run cross-validation on best hyperparameters
clf_cv_score = cross_val_score(clf, X_Train, Y_Train, cv=8)

print(clf_cv_score)
print('\n')

print(" - Decision Tree: ",clf_cv_score.mean())

#test data
clf_predict_Test=clf.predict(X_Test)

pred=pd.DataFrame(clf_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_HY_DT.csv',index=None)

{'min_samples_split': 40, 'min_samples_leaf': 3}
RMSE (training) for Decision Tree:726892734.764683
[0.77603488 0.8390646  0.81688391 0.75771187 0.80051729 0.71030441
 0.75753323 0.37513411]


 - Decision Tree:  0.7291480356599307
       SalePrice
0  130500.000000
1  157702.380952
2  178454.166667
3  164250.000000
4  211136.605263
     Id      SalePrice
0  1461  130500.000000
1  1462  157702.380952
2  1463  178454.166667
3  1464  164250.000000
4  1465  211136.605263


In [60]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestRegressor()
rfc.fit(X_Train,Y_Train)
rfc_predict_train=rfc.predict(X_Train)

mean_squared_error(Y_Train,rfc_predict_train)
print("RMSE (training) for random forest :{0:10f}".format(mean_squared_error(Y_Train,rfc_predict_train)))


#test data
rfc_predict_Test=rfc.predict(X_Test)

pred=pd.DataFrame(rfc_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_DF_RF.csv',index=None)


RMSE (training) for random forest :150856372.569904
   SalePrice
0   128640.0
1   158725.0
2   170498.5
3   175348.5
4   201590.0
     Id  SalePrice
0  1461   128640.0
1  1462   158725.0
2  1463   170498.5
3  1464   175348.5
4  1465   201590.0


In [61]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,100,10),'min_samples_split' : range(10,100,10),'min_samples_leaf': range(2,8,1)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=10)
rfc_random.fit(X_Train,Y_Train)
rand_parm_rfc=rfc_random.best_params_
print(rand_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestRegressor(**rand_parm_rfc)
rfc.fit(X_Train,Y_Train)
rfc_predict_train = rfc.predict(X_Train)


mean_squared_error(Y_Train,rfc_predict_train)
print("RMSE (training) for random forest:{0:10f}".format(mean_squared_error(Y_Train,rfc_predict_train)))

#run cross-validation on best hyperparameters
rfc_cv_score = cross_val_score(rfc, X_Train, Y_Train, cv=8)

print(rfc_cv_score)
print('\n')

print(" - rf: ",rfc_cv_score.mean())

#test data
rfc_predict_Test=rfc.predict(X_Test)

pred=pd.DataFrame(rfc_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_HY_RF.csv',index=None)

{'n_estimators': 70, 'min_samples_split': 20, 'min_samples_leaf': 7}
RMSE (training) for random forest:595095349.881966
[0.87239782 0.8891003  0.8299478  0.84040296 0.85890502 0.86072719
 0.80903069 0.74759986]


 - rf:  0.8385139555186865
       SalePrice
0  126907.215082
1  152577.208231
2  181653.314517
3  183502.495741
4  201038.743894
     Id      SalePrice
0  1461  126907.215082
1  1462  152577.208231
2  1463  181653.314517
3  1464  183502.495741
4  1465  201038.743894


In [62]:
#gradient boosting default

grd =GradientBoostingRegressor()
grd.fit(X_Train, Y_Train)
grd_predict_train=grd.predict(X_Train)


mean_squared_error(Y_Train,grd_predict_train)
print("RMSE (training) for gradient bossting:{0:10f}".format(mean_squared_error(Y_Train,grd_predict_train)))

#test data
grd_predict_Test=grd.predict(X_Test)

pred=pd.DataFrame(grd_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_DF_GB.csv',index=None)#

RMSE (training) for gradient bossting:212462549.487047
       SalePrice
0  120798.990284
1  164734.131316
2  176371.812607
3  180326.012793
4  197067.611035
     Id      SalePrice
0  1461  120798.990284
1  1462  164734.131316
2  1463  176371.812607
3  1464  180326.012793
4  1465  197067.611035


In [63]:
#Hyperparameter tuning for gb

parameters={ 'max_depth': [100,150],'learning_rate': [0.1,0.2]}
grd_random = RandomizedSearchCV(grd,parameters,n_iter=10)
grd_random.fit(X_Train, Y_Train)
rand_parm_grd=grd_random.best_params_
print(rand_parm_grd)

#contruct gb using the best parameters
grd= GradientBoostingRegressor(**rand_parm_grd)
grd.fit(X_Train, Y_Train)
grd_predict_train=grd.predict(X_Train)

mean_squared_error(Y_Train,grd_predict_train)
print("RMSE (training) for gradient bossting:{0:10f}".format(mean_squared_error(Y_Train,grd_predict_train)))

#run cross-validation on best hyperparameters
grd_cv_score = cross_val_score(grd, X_Train, Y_Train, cv=8)

print(grd_cv_score)
print('\n')

print(" - gb: ",grd_cv_score.mean())

#test data
grd_predict_Test=grd.predict(X_Test)

pred=pd.DataFrame(grd_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_HY_GB.csv',index=None)

{'max_depth': 150, 'learning_rate': 0.2}
RMSE (training) for gradient bossting:  0.000000
[0.79229852 0.74572945 0.74795422 0.82875429 0.81777947 0.68404416
 0.78335061 0.65521973]


 - gb:  0.7568913047231297
       SalePrice
0  128779.121806
1  155790.517691
2  190108.919395
3  180220.035439
4  213980.852274
     Id      SalePrice
0  1461  128779.121806
1  1462  155790.517691
2  1463  190108.919395
3  1464  180220.035439
4  1465  213980.852274


In [65]:
#MLP Regressor================================================================================
#default
nn =MLPRegressor()
nn.fit(X_Train, Y_Train)
nn_predict_train=nn.predict(X_Train)

mean_squared_error(Y_Train,nn_predict_train)
print("RMSE (training) for nn:{0:10f}".format(mean_squared_error(Y_Train,nn_predict_train)))

#test data
nn_predict_Test=nn.predict(X_Test)

pred=pd.DataFrame(nn_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_DF_NN.csv',index=None)#

RMSE (training) for nn:1961234276.561850
       SalePrice
0  145632.287629
1  173643.158952
2  194246.383664
3  195223.646045
4  170995.210099
     Id      SalePrice
0  1461  145632.287629
1  1462  173643.158952
2  1463  194246.383664
3  1464  195223.646045
4  1465  170995.210099


In [66]:
#Hyperparameter tuning for mlp
parameters={ 'learning_rate': ['invscaling', 'adaptive'], 'solver': ['lbfgs', 'sgd', 'adam']}
#
nn_random = RandomizedSearchCV(nn,parameters,n_iter=10)
nn_random.fit(X_Train, Y_Train)
rand_parm_nn=nn_random.best_params_
print(rand_parm_nn)

#contruct mlp using the best parameters
nn= MLPRegressor(**rand_parm_nn)
nn.fit(X_Train,Y_Train)
nn_predict_train=nn.predict(X_Train)

mean_squared_error(Y_Train,nn_predict_train)
print("RMSE (training) for nn:{0:10f}".format(mean_squared_error(Y_Train,nn_predict_train)))

#run cross-validation on best hyperparameters
nn_cv_score = cross_val_score(nn, X_Train, Y_Train, cv=8)

print(nn_cv_score)
print('\n')

print(" - nn: ",nn_cv_score.mean())

#test data
nn_predict_Test=nn.predict(X_Test)

pred=pd.DataFrame(nn_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_HY_NN.csv',index=None)#

{'solver': 'lbfgs', 'learning_rate': 'invscaling'}
RMSE (training) for nn:1320994854.449749
[ 0.75076941  0.71205408  0.74398796  0.80957464  0.73931377  0.71360936
  0.74138709 -0.17834072]


 - nn:  0.6290444482755345
       SalePrice
0  155760.299088
1  150305.342650
2  200988.891312
3  193145.846064
4  163782.269497
     Id      SalePrice
0  1461  155760.299088
1  1462  150305.342650
2  1463  200988.891312
3  1464  193145.846064
4  1465  163782.269497


In [67]:
#LinearSVR================================================================================

svr =LinearSVR()
svr.fit(X_Train, Y_Train)
svr_predict_train=svr.predict(X_Train)

mean_squared_error(Y_Train,svr_predict_train)
print("RMSE (training) for svr:{0:10f}".format(mean_squared_error(Y_Train,svr_predict_train)))


#test data
svr_predict_Test=svr.predict(X_Test)

pred=pd.DataFrame(svr_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_DF_SVR.csv',index=None)#

RMSE (training) for svr:1941998164.549139
       SalePrice
0  121998.877246
1  134289.173800
2  190875.355608
3  188690.613224
4  160291.406807
     Id      SalePrice
0  1461  121998.877246
1  1462  134289.173800
2  1463  190875.355608
3  1464  188690.613224
4  1465  160291.406807


In [68]:
#Hyperparameter tuning for svr
parameters={ 'loss': ['epsilon_insensitive','squared_epsilon_insensitive'], 'max_iter': [1000,2000]}
svr_random = RandomizedSearchCV(svr,parameters,n_iter=10)
svr_random.fit(X_Train, Y_Train)
rand_parm_svr=svr_random.best_params_
print(rand_parm_svr)

#contruct svr using the best parameters
svr= LinearSVR(**rand_parm_svr)
svr.fit(X_Train,Y_Train)
svr_predict_train=svr.predict(X_Train)

mean_squared_error(Y_Train,svr_predict_train)
print("RMSE (training) for svr:{0:10f}".format(mean_squared_error(Y_Train,svr_predict_train)))

#run cross-validation on best hyperparameters
svr_cv_score = cross_val_score(svr, X_Train, Y_Train, cv=8)

print(svr_cv_score)
print('\n')

print(" - svr: ",svr_cv_score.mean())

#test data
svr_predict_Test=svr.predict(X_Test)

pred=pd.DataFrame(svr_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_HY_SVR.csv',index=None)

{'max_iter': 2000, 'loss': 'squared_epsilon_insensitive'}
RMSE (training) for svr:2518162509.503254
[ 0.76240134 -0.08873826  0.63672986  0.51520354  0.64376572  0.49939853
  0.72517143 -1.37997343]


 - svr:  0.2892448401415142
       SalePrice
0  112338.699487
1  102349.702403
2  176529.219271
3  179321.151054
4  164153.019640
     Id      SalePrice
0  1461  112338.699487
1  1462  102349.702403
2  1463  176529.219271
3  1464  179321.151054
4  1465  164153.019640


In [69]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using Neural networks, RandomForest and Decision Tree Classifier\n")

models = [MLPRegressor(),  RandomForestRegressor(), DecisionTreeRegressor() ]#we r giving models as arguments we can also give those parameters like grid_params
      #we are giving  data after smote and we did smote only for train test
Strain, Stest = stacking(models,                   
                           X_Train, Y_Train, X_Test,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                         
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
                         
                         verbose=2)
         

___________________________________________________________________________________________
Ensemble Methods Predictions using Neural networks, RandomForest and Decision Tree Classifier

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [MLPRegressor]
    fold  0:  [32029.20334698]
    fold  1:  [31745.66233334]
    fold  2:  [35498.06863143]
    fold  3:  [27809.02728425]
    ----
    MEAN:     [31770.49039900] + [2722.84677822]
    FULL:     [31770.49039900]

model  1:     [RandomForestRegressor]
    fold  0:  [18729.63561644]
    fold  1:  [19413.53972603]
    fold  2:  [18447.00328767]
    fold  3:  [17768.71780822]
    ----
    MEAN:     [18589.72410959] + [590.05677259]
    FULL:     [18589.72410959]

model  2:     [DecisionTreeRegressor]
    fold  0:  [28155.82191781]
    fold  1:  [24985.18630137]
    fold  2:  [27324.64657534]
    fold  3:  [23408.21643836]
    ----
    MEAN:     [25968.46780822] + [188

In [70]:
#stacking one layer dt
clf = DecisionTreeRegressor()
clf.fit(Strain, Y_Train)
clf_predict_train=clf.predict(Strain)

#clf.feature_importances_
mean_squared_error(Y_Train,clf_predict_train)
print("RMSE (training) for Stacking Decision Tree:{0:10f}".format(mean_squared_error(Y_Train,clf_predict_train)))

#test data
clf_predict_Test=clf.predict(Stest)

pred=pd.DataFrame(clf_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_DF_DT_Stack.csv',index=None)

RMSE (training) for Stacking Decision Tree:  0.000000
   SalePrice
0   126500.0
1   155000.0
2   189000.0
3   176000.0
4   210000.0
     Id  SalePrice
0  1461   126500.0
1  1462   155000.0
2  1463   189000.0
3  1464   176000.0
4  1465   210000.0


In [71]:
#Hyperparameter tuning done for decision tree 
parameters={'min_samples_split' : range(10,100,10),'min_samples_leaf': range(2,8,1)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=10)
clf_random.fit(Strain,Y_Train)
rand_parm=clf_random.best_params_
print(rand_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTree 
clf = DecisionTreeRegressor(**rand_parm)
clf.fit(Strain,Y_Train)
clf_predict_train = clf.predict(Strain)

#clf.feature_importances_
mean_squared_error(Y_Train,clf_predict_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(Y_Train,clf_predict_train)))

#run cross-validation on best hyperparameters
clf_cv_score = cross_val_score(clf, Strain, Y_Train, cv=8)

print(clf_cv_score)
print('\n')

print(" - Decision Tree: ",clf_cv_score.mean())

#test data
clf_predict_Test=clf.predict(Stest)

pred=pd.DataFrame(clf_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_HY_DT_Stack.csv',index=None)

{'min_samples_split': 40, 'min_samples_leaf': 7}
RMSE (training) for Decision Tree:735872560.067062
[0.86343073 0.83056788 0.83448509 0.82231782 0.88078944 0.86274962
 0.78102614 0.70587683]


 - Decision Tree:  0.8226554442028952
       SalePrice
0  121820.000000
1  166782.000000
2  182562.187500
3  182562.187500
4  203393.777778
     Id      SalePrice
0  1461  121820.000000
1  1462  166782.000000
2  1463  182562.187500
3  1464  182562.187500
4  1465  203393.777778


In [72]:
#gradient boosting default

grd =GradientBoostingRegressor()
grd.fit(Strain, Y_Train)
grd_predict_train=grd.predict(Strain)


mean_squared_error(Y_Train,grd_predict_train)
print("RMSE (training) for gradient bossting:{0:10f}".format(mean_squared_error(Y_Train,grd_predict_train)))

#test data
grd_predict_Test=grd.predict(Stest)

pred=pd.DataFrame(grd_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_DF_GB_Stack.csv',index=None)#

RMSE (training) for gradient bossting:411289373.255711
       SalePrice
0  125374.196919
1  161523.559333
2  184833.377830
3  184216.454843
4  202423.319480
     Id      SalePrice
0  1461  125374.196919
1  1462  161523.559333
2  1463  184833.377830
3  1464  184216.454843
4  1465  202423.319480


In [73]:
#Hyperparameter tuning for gb

parameters={ 'max_depth': [100,150],'learning_rate': [0.1,0.2]}
grd_random = RandomizedSearchCV(grd,parameters,n_iter=10)
grd_random.fit(Strain, Y_Train)
rand_parm_grd=grd_random.best_params_
print(rand_parm_grd)

#contruct gb using the best parameters
grd= GradientBoostingRegressor(**rand_parm_grd)
grd.fit(Strain, Y_Train)
grd_predict_train=grd.predict(Strain)

mean_squared_error(Y_Train,grd_predict_train)
print("RMSE (training) for gradient bossting:{0:10f}".format(mean_squared_error(Y_Train,grd_predict_train)))

#run cross-validation on best hyperparameters
grd_cv_score = cross_val_score(grd, Strain, Y_Train, cv=8)

print(grd_cv_score)
print('\n')

print(" - gb: ",grd_cv_score.mean())

#test data
grd_predict_Test=grd.predict(Stest)

pred=pd.DataFrame(grd_predict_Test,columns=["SalePrice"])
print(pred.head())
output_col=pd.concat([testData["Id"],pred],axis=1) 
print(output_col.head())
output_col.to_csv('/gdrive/My Drive/CIS508/TeamAssignment1/Colab/Result_HY_GB_Stack.csv',index=None)

{'max_depth': 100, 'learning_rate': 0.2}
RMSE (training) for gradient bossting:  0.000000
[0.83953551 0.76060924 0.82962043 0.76542931 0.74392831 0.74809172
 0.78666867 0.75196876]


 - gb:  0.7782314940446271
       SalePrice
0  126500.000001
1  155000.000845
2  188024.260119
3  177772.772363
4  209999.999272
     Id      SalePrice
0  1461  126500.000001
1  1462  155000.000845
2  1463  188024.260119
3  1464  177772.772363
4  1465  209999.999272
